# Early-Stopping für Softmax-Regression
Implementieren sie das Batch-Gradientenverfahren mit Early Stopping für die Softmax-Regression (ohne Scikit-Learn)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
from sklearn.datasets import load_iris

iris = load_iris()

In [12]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [85]:
# [classes]
y = iris['target']
print(y.shape)
y[:3]

(150,)


array([0, 0, 0])

In [86]:
# [['petal length', 'petal width']]
X = iris['data'][:, (2,3)]
print(X.shape)
X[:3]

(150, 2)


array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2]])

In [87]:
eta = 0.1
n_iter = 1000
m = len(X)

**Softmax-Score für Kategorie k**  
$$s_k(x) = (\theta^{(k)})^T*x$$

In [182]:
theta = np.random.randn(3, 2)
print(theta.shape)
theta

(3, 2)


array([[ 1.19316014,  1.02315672],
       [-0.18731305,  1.31154586],
       [ 0.58644648, -0.31855708]])

In [92]:
X.shape

(150, 2)

In [93]:
#softmax-score für jeden datenpunkt und jede kategorie
# [
# [cat1, cat2], -> erster Datenpunkt
# [cat1, cat2] -> zweiter Datenpunkt
#]
score = []
for row in X:
    row_score = []
    for cat in theta:
        cat_score = cat.T.dot(row)
        row_score.append(cat_score)
    score.append(row_score)
score = np.array(score)
print(score.shape)
score[:2]

(150, 3)


array([[ 0.15274993, -0.64130152,  0.23222408],
       [ 0.15274993, -0.64130152,  0.23222408]])

**Softmax-Funktion**  
$$\hat p_k = \sigma(s(x())_k = \frac{exp(s_k(x))}{\sum_{j=1}^Kexp(s_j(x))}$$
* $K$ ist die Anzahl der Kategorien
* $s(x)$ ist ein Vektor der die Scores jeder Kategorie für den Datenpunkt x enthält
* $\sigma(s(x))_k$ ist die anhand der Sores geschätze Wahrscheinlichkeit dafür, dass der Datenpunkt $x$ zur Kategorie $k$ gehört.

In [160]:
softmax = []
for p in score:
    p_softmax = []
    cat_score_sum = np.sum([np.exp(cat_score) for cat_score in p])
    for cat_score in p:
        cat_softmax = np.exp(cat_score) / cat_score_sum
        p_softmax.append(cat_softmax)
    softmax.append(p_softmax)
softmax = np.array(softmax)
print(softmax.shape)
softmax[:2]

(150, 3)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


array([[ 0., nan,  0.],
       [ 0., nan,  0.]])

**Vorhersage eines Klassifikators mit Softmax-Regression**  
$$\hat y = argmax \sigma(s(x))_k = argmax s_k(x) = argmax ((\theta^{(k)})^T * x))$$
* Der Operator argmax liefert den Wert einer Variable, der eine Funktion maximiert. Hier liefert er denjenigen Wert für $k$, für den die geschätzte Wahrscheinlichkeit $\sigma(s(x))_k$ maximal wird

In [98]:
probas = []
for proba in softmax:
    probas.append(np.argmax(proba))
probas = np.array(probas)
print(probas.shape)
probas[:2]

(150,)


array([2, 2])

**Kreuzentropie als Kostenfunktion**  
$$J(\Theta) = -\frac{1}{m}\sum_{i=1}^m\sum_{k=1}y_k^{(i)}log(\hat p_k^{(i)})$$
* $y_k^{(i)}$ ist gleich 1, wenn die Zielkategorie des $i^{ten}$ Datenpunkts $k$ ist, sonst 0.

**Gradientenvektor der Kreuzentropie für Kategorie k**  
$$\nabla_{\theta}J(\Theta) = \frac{1}{m}\sum_{i=1}^m(\hat p_k^{(i)} - y_k^{(i)})x^{(i)}$$

In [118]:
print(theta.shape, probas.shape, softmax.shape, score.shape, X.shape, y.shape, sep='\n')

(3, 2)
(150,)
(150, 3)
(150, 3)
(150, 2)
(150,)


In [134]:
one = y == 0
two = y == 1
three = y == 2
y_new = np.array([one,two,three]).astype(np.int).T
print(y_new.shape)
y_new[:3]

(150, 3)


array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [151]:
# Training
for epoch in range(1000):
    for i, cat_theta in enumerate(theta):
        bla = 1 / m * np.sum((softmax[:,i] - y_new[:,i]).dot(X))
        theta[i] = cat_theta - eta * bla

In [152]:
theta

array([[-557.97624485, -556.32278686],
       [ 705.74726961,  704.00753978],
       [-148.1389498 , -146.39091518]])

# Hier beginnt das richtige!!

In [164]:
# Softmax-Score für jede Kategorie und jeden Datenpunkt
def get_score(theta, X):
    score = []
    for cat in theta:
        cat_score = []
        for row in X:
            row_score = cat.T.dot(row)
            cat_score.append(row_score)
        score.append(cat_score)
    score = np.array(score).T
    print(score.shape)
    return score

In [166]:
get_score(theta, X)[:3]

(150, 3)


array([[-892.43130016, 1128.84768541, -236.67271275],
       [-892.43130016, 1128.84768541, -236.67271275],
       [-836.63367568, 1058.27295845, -221.85881777]])

In [170]:
# Softmax-Funktion
def get_softmax(score):
    softmax = []
    for p in score:
        p_softmax = []
        cat_score_sum = np.sum([np.exp(cat_score) for cat_score in p])
        for cat_score in p:
            cat_softmax = np.exp(cat_score) / cat_score_sum
            p_softmax.append(cat_softmax)
        softmax.append(p_softmax)
    softmax = np.array(softmax)
    print(softmax.shape)
    return softmax

In [179]:
get_softmax(get_score(theta, X))[:3]

(150, 3)
(150, 3)


array([[0.05388868, 0.93468103, 0.01143029],
       [0.05388868, 0.93468103, 0.01143029],
       [0.06695002, 0.91757013, 0.01547985]])

In [180]:
# Prediction
def predict(softmax):
    probas = []
    for proba in softmax:
        probas.append(np.argmax(proba))
    probas = np.array(probas)
    print(probas.shape)
    return probas

In [246]:
predict(get_softmax(get_score(theta, X)))

(150, 3)
(150, 3)
(150,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [222]:
# Training
# TODO: softmax muss als parameter übergeben werden!!!
def fit(theta, X, y, eta=0.1, n_iter=100):
    for epoch in range(n_iter):
        for i, cat_theta in enumerate(theta):
            bla = 1 / m * np.sum((softmax[:,i] - y[:,i]).dot(X))
            theta[i] = cat_theta - eta * bla

In [244]:
fit(theta, X, y_new)
theta.shape

(3, 2)

In [245]:
theta

array([[115.05982681, 114.88982339],
       [         nan,          nan],
       [505.78644648, 504.88144292]])

Reihenfolge
1. score berechnen
2. softmax funktion berechnen
3. trainieren
4. validieren
5. trainieren
6. validieren
7. ...
8. testen